<a href="https://colab.research.google.com/github/snipaid-nlg/models/blob/main/news-headline-generation/news-headline-generation-with-bloom-1b5-clp-german.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# News Headline Generation with bloom-1b5-clp-german
Evaluating bloom-1b5-clp-german on the task of generating headlines from journalistic text in german.

## Installs and Imports

In [ ]:
!pip install -q transformers datasets accelerate sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 KB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 751.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset, Dataset
from transformers.pipelines.pt_utils import KeyDataset
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import pandas as pd
from tqdm.auto import tqdm

## Loading and transforming 10kGNAD
We load the [Ten Thousand German News Articles Dataset](https://tblock.github.io/10kGNAD/) from Huggingface. The first sentence of a text (up to the first period) is a headline, so we split the text and use the second part. A Suffix `Headline:` is added to prompt the model.

In [ ]:
gnad10 = load_dataset("gnad10")
df = pd.DataFrame(gnad10["train"])
df[['headline', 'fulltext']] = df["text"].str.split('.', n=1, expand=True)
df.drop(['label','text'], axis=1, inplace=True)
df['fulltext'] = df['fulltext'] + " Headline:"
df.head()

Generating train split:   0%|          | 0/9245 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1028 [00:00<?, ? examples/s]

Dataset gnad10 downloaded and prepared to /root/.cache/huggingface/datasets/gnad10/default/1.1.0/3a8445be65795ad88270af4d797034c3d99f70f8352ca658c586faf1cf960881. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

,headline,fulltext
0,21-Jähriger fällt wohl bis Saisonende aus,Wien – Rapid muss wohl bis Saisonende auf Off...
1,"Erfundene Bilder zu Filmen, die als verloren g...",Wien – Die Filmgeschichte ist ein Friedhof de...
2,Der frischgekürte CEO Sundar Pichai setzt auf ...,Die Atmosphäre im Silicon Valley ist rau. Da ...
3,"Putin: ""Einigung, dass wir Menge auf Niveau vo...",Moskau – Die russischen Ölproduzenten wollen ...
4,Estland sieht den künftigen österreichischen P...,Wien/Tallinn/Pskow – Die Eltern des künftigen...


Converting the pandas dataframe to a Huggingface dataset object

In [ ]:
dataset = Dataset.from_pandas(df)

## Loading the model

In [ ]:
model = AutoModelForCausalLM.from_pretrained("malteos/bloom-1b5-clp-german", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("malteos/bloom-1b5-clp-german")

In [ ]:
template = {
      "min_new_tokens": 5,
      "max_new_tokens": 20,
      "length_penalty": 0.5,
      "num_beams": 3,
      "repetition_penalty": 1.5,
      "num_beam_groups": 1,
      "do_sample": False,
      "top_k": 50,
      "top_p": 0.7,
      "temperature": 1.0,
      "early_stopping": True,
      "pad_token_id": 0,
      "eos_token_id": 186, # Token: \n
      "num_return_sequences": 1,
      "return_full_text": False,
}

In [ ]:
pipe = pipeline("text-generation", 
                model=model,
                tokenizer=tokenizer,
                device=0,
                **template)

In [ ]:
for out in tqdm(pipe(KeyDataset(dataset, "fulltext"))):
  print(out)

  0%|          | 0/9245 [00:00<?, ?it/s]

[{'generated_text': ': „Rapid muss wohl bis Saisonende auf Offensivspieler Thomas Murg verzichten.“\n'}]
[{'generated_text': ': The Forbidden Room\n'}]
[{'generated_text': '-Meldungen wie „Android-Chef Pichai tritt zurück“ oder „Pichai'}]
[{'generated_text': ': „Russische Ölproduzenten wollen Produktion einfrieren“ weiterlesen\n'}]
[{'generated_text': ': „Van der Bellens Sohn wird neuer Bundespräsident“\n'}]
[{'generated_text': ': Trotec übernimmt US-Unternehmen\n'}]
[{'generated_text': ': Heracles Almelo – FC Utrecht\n'}]
[{'generated_text': ': "ORF-Generaldirektor Wrabetz: Keine Eile beim Funkhaus-Verkauf" ('}]
[{'generated_text': ': Pakistanische Luftwaffe tötet 43 Extremisten\n'}]


KeyboardInterrupt: ignored